# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
#from matplotlib_venn import venn2

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject

# importing the csv-file of data on share price index, short-term interest rates, long-term interest rates etc. 
#interest = pd.read_excel('02024c3c-3d9b-4ea1-a818-c73cf2d0e0c1.xlsx')
interest = pd.read_csv('MEI_FIN_05042023142240461.csv')
interest.head()

# Filtering, so only the long-term interest rates appear
df = interest[interest.Subject =="Long-term interest rates, Per cent per annum"]

# resetting the index
df.index = range(df.shape[0])

df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,SUBJECT,Subject,LOCATION,Country,FREQUENCY,Frequency,TIME,Time,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,IRLT,"Long-term interest rates, Per cent per annum",CAN,Canada,A,Annual,1974,1974,PC,Percentage,0,Units,NaN,NaN,8.870000,NaN,NaN
1,IRLT,"Long-term interest rates, Per cent per annum",CAN,Canada,A,Annual,1975,1975,PC,Percentage,0,Units,NaN,NaN,8.993333,NaN,NaN
2,IRLT,"Long-term interest rates, Per cent per annum",CAN,Canada,A,Annual,1976,1976,PC,Percentage,0,Units,NaN,NaN,9.229167,NaN,NaN
3,IRLT,"Long-term interest rates, Per cent per annum",CAN,Canada,A,Annual,1977,1977,PC,Percentage,0,Units,NaN,NaN,8.693334,NaN,NaN
4,IRLT,"Long-term interest rates, Per cent per annum",CAN,Canada,A,Annual,1978,1978,PC,Percentage,0,Units,NaN,NaN,9.235833,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,IRLT,"Long-term interest rates, Per cent per annum",CHN,China (People's Republic of),A,Annual,2018,2018,PC,Percentage,0,Units,NaN,NaN,3.579167,NaN,NaN
272,IRLT,"Long-term interest rates, Per cent per annum",CHN,China (People's Republic of),A,Annual,2019,2019,PC,Percentage,0,Units,NaN,NaN,3.181300,NaN,NaN
273,IRLT,"Long-term interest rates, Per cent per annum",CHN,China (People's Republic of),A,Annual,2020,2020,PC,Percentage,0,Units,NaN,NaN,2.924433,NaN,NaN
274,IRLT,"Long-term interest rates, Per cent per annum",CHN,China (People's Republic of),A,Annual,2021,2021,PC,Percentage,0,Units,NaN,NaN,3.005533,NaN,NaN


In [11]:
# counting how many values there is for each country
interest.Country.value_counts()


United States                   196
Canada                          195
United Kingdom                  180
Germany                         147
Netherlands                     139
Korea                           128
China (People's Republic of)     98
Name: Country, dtype: int64

In [12]:
# Creating a new dataframe, where we filter out some the values we do not care about. 
dataframe = pd.DataFrame(df, columns = ['SUBJECT', 'Country', 'Time', 'Value'])
print(dataframe)

    SUBJECT                       Country  Time     Value
0      IRLT                        Canada  1974  8.870000
1      IRLT                        Canada  1975  8.993333
2      IRLT                        Canada  1976  9.229167
3      IRLT                        Canada  1977  8.693334
4      IRLT                        Canada  1978  9.235833
..      ...                           ...   ...       ...
271    IRLT  China (People's Republic of)  2018  3.579167
272    IRLT  China (People's Republic of)  2019  3.181300
273    IRLT  China (People's Republic of)  2020  2.924433
274    IRLT  China (People's Republic of)  2021  3.005533
275    IRLT  China (People's Republic of)  2022  2.764183

[276 rows x 4 columns]


# Read and clean data

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [ ]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [ ]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.